# A. CONTENT BASE FILTERING METHOD

# B. IMPORT LIB

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset

import warnings; warnings.simplefilter('ignore')
# 1. Import library
import lib.step1ml_utilis_summaryPre_processing as sum
import lib.step2ml_utils_feature_selection as feature
import lib.step3ml_utils_model_design_testing_classification as classi
import lib.step4ml_utils_model_design_testing_explainability as tests
import lib.step5ml_utils_model_design_testing_visualize_models as visu
import lib.step6ml_utils_model_design_testing_geospatial_analysis as ana
import lib.step7ml_utilis_NLP_Vietnamese as nlp
import lib.Step14_NLP_vietnamese as nlp_vietnamese
import lib.step8ml_utilis_NLP as nlp_model
import lib.step9ml_Text_Kmeans_Clustering as pys

import warnings
warnings.filterwarnings("ignore")

# C. MODEL

## 1.1. Upload data

In [16]:
df = pd.read_csv('data_analysis/ProductRaw_processing_3.csv', encoding='utf-8')
data = df.copy()
data.head()

,Unnamed: 0,item_id,name,description,rating,price,list_price,brand,group,url,image,fea_item_id,fea_name,fea_name_2,fea_description,fea_rating,fea_price,fea_list_price,fea_brand,fea_group
0,0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.00,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,48102821,tai nghe bluetooth inpods 12 cảm_biến vân tay ...,tai bluetooth inpods 12 cảm_biến vân chống nướ...,thông_tin chi_tiết dung_lượng pin 3 thời_gian ...,4.00,77000,300000,oem,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
1,1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.50,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...,52333193,tai nghe bluetooth không dây f true wireless d...,tai bluetooth không dây f true wireless dock s...,thông_tin chi_tiết dung_lượng pin 2 mah thời_g...,4.50,132000,750000,oem,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
2,2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.80,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...,299461,chuột không dây logitech m331 silenthàng chính...,chuột không dây logitech m331 silenthàng hãng,thông_tin chi_tiết thương_hiệu logitech độ phâ...,4.80,299000,399000,logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...
3,3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.70,149000,350000,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...,57440329,loa bluetooth 5 kiêm đồng_hồ báo_thức 2 trong ...,loa bluetooth 5 kiêm đồng_hồ báo_thức 2 1 robo...,thông_tin chi_tiết thương_hiệu acome xuất_xứ t...,4.70,149000,350000,acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...
4,4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.80,5090000,8500000,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...,38458616,tai nghe bluetooth apple airpods pro true wire...,tai bluetooth apple airpods pro true wirelessm...,thông_tin chi_tiết thương_hiệu apple xuất_xứ t...,4.80,5090000,8500000,apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...


## 1.2. New data

In [113]:
lst = ['item_id', 'name', 'description', 'rating', 'image', 'url', 'price', 'list_price', 'brand']
data_new = data[lst]

In [114]:
data_new.loc[data_new['description'].isnull(),'fea_description'] = 'Trống'

In [115]:
lst = ['item_id', 'name', 'description', 'rating', 'image', 'url', 'price', 'list_price', 'brand']
data_content = data_new[lst]

In [20]:
data_content

,item_id,name,description,rating,image,url
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.00,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.50,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tai-nghe-bluetooth-khong-day-f9-true-w...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.80,https://salt.tikicdn.com/cache/280x280/media/c...,https://chuot-khong-day-logitech-m331-silent-p...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.70,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.80,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tai-nghe-bluetooth-apple-airpods-pro-t...
5,12567795,Tai Nghe Bluetooth Nhét Tai Apple AirPods 2 Tr...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.80,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tai-nghe-bluetooth-nhet-tai-apple-airp...
6,4897521,Tai Nghe Nhét Tai JBL C150SI - Hàng Chính Hãng,THÔNG TIN CHI TIẾT\nThương hiệu JBL\nXuất xứ t...,4.50,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tai-nghe-nhet-tai-jbl-c150si-hang-chin...
7,47321729,Pin Sạc Dự Phòng Xiaomi Redmi 20000mAh PB200LM...,THÔNG TIN CHI TIẾT\nDung lượng pin 20000\nThươ...,4.70,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://pin-sac-du-phong-xiaomi-redmi-20000mah...
8,2638665,Tai nghe nhét tai BYZ S Super Bass cho IPHONE/...,THÔNG TIN CHI TIẾT\nThương hiệu Hoco\nModel S3...,4.70,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tai-nghe-nhet-tai-byz-s-super-bass-cho...
9,25078249,Chuột Không Dây Logitech M330 Silent Plus- Hàn...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nXuất...,4.70,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://chuot-khong-day-logitech-m330-silent-p...


In [21]:
data_content.name[1]

'Tai nghe bluetooth không dây F9 True wireless Dock Sạc có Led Báo Pin Kép'

In [40]:
data_content.description[1]

'THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nThời gian pin Thời gian nghe nhạc: 100 giờ Thời gian đàm thoại: 120 giờ\nBluetooth v5.0\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nĐộ nhạy cao\nModel F9\nLoại Jack cắm Không dây/Bluetooth\nTrọng lượng 300g\nTần số 25hz\nThời gian sử dụng\nThời gian nghe nhạc: 100 giờ\nThời gian đàm thoại: 120 giờ\nHỗ trợ đàm thoại Có\nSKU 7319492804496\nMÔ TẢ SẢN PHẨM\nSử dụng Chip Bluetooth 5.0 mạnh mẽ cho kết nối ổn định, nhanh hơn với khoảng cách kết nối xa (10m) và tiết kiệm năng lượng (Pin)\nTự khỏi động khi nhấc ra khỏi hộp sạc\nTự động kết nối Bluetooth với điện thoại khi ở trạng thai sẵn sàng.\nTự ngắt kế nối khi bỏ vào hộp sạc\nKhoảng cách hoạt động: phạm vi 10m\nCông nghệ khử tiếng ồn cho âm thanh trung thực nhất và chất lượng cao (CVC 8.0)\nDung lượng pin từng tai: 50mAh\nTrọng lượng tai nghe: 4g (rất nhẹ và khít cho cảm giác thoải mái)\nDung lương Dock sạc: 2000 mAh, có thể sạc cho tai nghe 20 lần. Không chỉ sạc cho tai nghe mà còn có thể 

## 1.3. NLP

In [22]:
fea_content_2 = nlp_vietnamese.nlp_vietnamese(df=data_content, col_name='description', columns=['fea_descriptiont_2'])
fea_content_2

,fea_descriptiont_2
0,thông liên_tục sạc chờ thương_hiệu xuất_xứ quố...
1,thông đàm_thoại thương_hiệu xuất_xứ quốc nhạy ...
2,thông logitech học lưu_ý kéo_dài chuột làm_việ...
3,thông thương_hiệu xuất_xứ xuất_xứ sku sản_phẩm...
4,thông thương_hiệu thương_hiệu kích_thước sạc s...
5,thông xuất_xứ mỹ quốc mô_tả sạc sạc mã phiên_b...
6,thông jbl thương_hiệu nhạy cắm quốc mô_tả tần_...
7,thông thương_hiệu xuất_xứ quốc xuất_xứ sku sản...
8,thông thương_hiệu trắng xuất_xứ mô_tả cắm máy_...
9,thông thương_hiệu xuất_xứ thụy sku sản_phẩm kh...


In [23]:
# Stop_word những từ không cần thiết ra khỏi dữ liệu
fea_content_3 = nlp.txt_stop_word(df=fea_content_2, lst_text='fea_descriptiont_2', columns_='fea_descriptiont_3')
fea_content_3 = fea_content_2.dropna()
fea_content_3

,fea_descriptiont_2
0,thông liên_tục sạc chờ thương_hiệu xuất_xứ quố...
1,thông đàm_thoại thương_hiệu xuất_xứ quốc nhạy ...
2,thông logitech học lưu_ý kéo_dài chuột làm_việ...
3,thông thương_hiệu xuất_xứ xuất_xứ sku sản_phẩm...
4,thông thương_hiệu thương_hiệu kích_thước sạc s...
5,thông xuất_xứ mỹ quốc mô_tả sạc sạc mã phiên_b...
6,thông jbl thương_hiệu nhạy cắm quốc mô_tả tần_...
7,thông thương_hiệu xuất_xứ quốc xuất_xứ sku sản...
8,thông thương_hiệu trắng xuất_xứ mô_tả cắm máy_...
9,thông thương_hiệu xuất_xứ thụy sku sản_phẩm kh...


In [26]:
# Lấy dữ liệu theo số lần lặp để thực hiện lấy những cụm từ có ý nghĩa trong bài toán của chúng ta
word_count = nlp.groupby_word_txt(df=fea_content_3, lst_text='fea_descriptiont_2', columns_='content_count')
word_count

,Num
content_count,
sạc,99
giúp,87
thông,58
phát,49
chống,48
...,...
lăn,1
lýtưởng,1
láng,1


## 1.4. Thực hiện model

### 1.4.1. TF-IDF Vectors
Term frequency-inverse document frequency is a statistic that reflects how important a word is to a specific document relative to all of the words in a collection of documents (the corpus). The tf-idf value increases proportionally to the number of times that word appears in the document, but is offset by the frequency of the word in the corpus.

Below, we use a TF-IDF vectorizer to determine the relative importance of the words in the first item description. Unlike the count vectorizer, this one requires the entire corpus as its input, so it takes a little bit longer to run. As you can see, the words with the highest TF-IDF values are now a bit more specific to first item, including words such as "brite", "xtreme", and "gel". 

In [27]:
STOP_WORD_FILE = 'data/vietnamese-stopwords.txt'
with open(STOP_WORD_FILE,'r',encoding='utf-8') as file: 
    stop_words = file.read()
stop_words = stop_words.split('\n')

In [28]:
#Define the TFIDF vectorizer that will be used to process the data
tfidf_vectorizer = TfidfVectorizer(analyzer='word',min_df=0,stop_words=stop_words)
#Apply this vectorizer to the full dataset to create normalized vectors
tfidf_matrix = tfidf_vectorizer.fit_transform(fea_content_2)
#Get the names of the features
features = tfidf_vectorizer.get_feature_names()

### 1.4.2. Using Cosine Similarity Score to Identify Similar items
- Let us first try to build a recommender using descriptions only

In [30]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(fea_content_3['fea_descriptiont_2'])
tfidf_matrix

<39x2568 sparse matrix of type '<class 'numpy.float64'>'
	with 4157 stored elements in Compressed Sparse Row format>

In [31]:
# how many keys words do we count overall
max(tfidf_matrix[[0]])

<1x2568 sparse matrix of type '<class 'numpy.float64'>'
	with 63 stored elements in Compressed Sparse Row format>

Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score. 

In [32]:
vector = cosine_similarity(X = tfidf_matrix, Y=None, dense_output=True)
vector

array([[1.        , 0.11133263, 0.03237064, ..., 0.02468557, 0.02394881,
        0.02652862],
       [0.11133263, 1.        , 0.0375789 , ..., 0.10400663, 0.02171338,
        0.03353029],
       [0.03237064, 0.0375789 , 1.        , ..., 0.08265942, 0.09575697,
        0.01917603],
       ...,
       [0.02468557, 0.10400663, 0.08265942, ..., 1.        , 0.05124999,
        0.03581466],
       [0.02394881, 0.02171338, 0.09575697, ..., 0.05124999, 1.        ,
        0.05845085],
       [0.02652862, 0.03353029, 0.01917603, ..., 0.03581466, 0.05845085,
        1.        ]])

We can also use sklearn's linear_kernel instead of cosine_similarities since it is much faster.

In [33]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [34]:
cosine_sim[1]

array([0.11133263, 1.        , 0.0375789 , 0.08102127, 0.10726049,
       0.08307432, 0.0393785 , 0.04339967, 0.0298513 , 0.05341162,
       0.0704936 , 0.04046165, 0.15920565, 0.03330238, 0.0221008 ,
       0.15846861, 0.05525859, 0.01276551, 0.02377869, 0.08072961,
       0.03705192, 0.10642692, 0.05620635, 0.76049088, 0.05455134,
       0.06863209, 0.02679644, 0.17269559, 0.04153208, 0.07607424,
       0.1535629 , 0.08748131, 0.0174716 , 0.03765723, 0.0043222 ,
       0.16716451, 0.10400663, 0.02171338, 0.03353029])

### 1.4.3. We now have a pairwise cosine similarity matrix for all the items in our dataset. 

In [35]:
product = fea_content_2.reset_index()
titles = product['fea_descriptiont_2']
indices = pd.Series(product.index, index=product['fea_descriptiont_2'])

In [36]:
# get title name
indices.head()

fea_descriptiont_2
thông liên_tục sạc chờ thương_hiệu xuất_xứ quốc nhạy_cảm_biến cắm liên_tục sản_phẩm tỉ_lệ chuẩn bấm thuận_tiện dễ_dàng vô_cùng vô_cùng kết_nối chất thời_lượng cải_thiện dock tiện_lợi sạc tương_thích oppo tablet liên_tục sạc chờ tuỳ phương_thức địa_chỉ phát vận_chuyển phụ_phí                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    0
thông đàm_thoại thương_hiệu xuất_xứ quốc nhạy cắm trọng_lượn

### 1.4.4. Create a function that takes a single row of the tf-idf matrix (corresponding to a particular document), and return the n highest scoring words (or more generally tokens or features):

In [37]:
def get_highest_cosine_sim(title):
    # get index of a particular item
    idx = indices[title]
    # list score of each title
    sim_scores = list(enumerate(cosine_sim[idx]))
    # sort scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # get 30 highest scores exclude itself
    sim_scores = sim_scores[1:31]
    # print(sim_scores)
    # get item index
    item_indices = [i[0] for i in sim_scores]
    item_distance = [j[1] for j in sim_scores]
    result = pd.DataFrame({'distance':item_distance, 'title': titles.iloc[item_indices]})
    return result

In [42]:
get_highest_cosine_sim('thông liên_tục sạc chờ thương_hiệu xuất_xứ quốc nhạy_cảm_biến cắm liên_tục sản_phẩm tỉ_lệ chuẩn bấm thuận_tiện dễ_dàng vô_cùng vô_cùng kết_nối chất thời_lượng cải_thiện dock tiện_lợi sạc tương_thích oppo tablet liên_tục sạc chờ tuỳ phương_thức địa_chỉ phát vận_chuyển phụ_phí').head(10)

,distance,title
24,0.20,thông lanith thương_hiệu sku sản_phẩm tws khôn...
1,0.11,thông đàm_thoại thương_hiệu xuất_xứ quốc nhạy ...
23,0.11,thông thương_hiệu xuất_xứ quốc mô_tả amoi trải...
35,0.09,thông anker quốc mô_tả gọn tích_hợp thông hệ_t...
27,0.09,thông anker thương_hiệu xuất_xứ sku sản_phẩm g...
29,0.09,thông oem thương_hiệu nhạy cắm trọng_lượng mô_...
3,0.08,thông thương_hiệu xuất_xứ xuất_xứ sku sản_phẩm...
19,0.08,thông thương_hiệu xuất_xứ quốc xuất_xứ mô_tả c...
12,0.08,thông sạc eb quốc sạc sạc sku sản_phẩm afc qui...
15,0.08,thông thương_hiệu xuất_xứ sku sản_phẩm baseus ...


In [45]:
data_content.columns

Index(['item_id', 'name', 'description', 'rating', 'image', 'url'], dtype='object')

In [139]:
def recommend(content):
    content=data_content[data_content.name==content].index.values[0]
    
    df_recommnend=pd.DataFrame(
        enumerate(vector[content])).drop(0,axis='columns').sort_values(by=1,ascending=False)
    
    df_recommnend['Names']=list(map(
        lambda x: str(np.squeeze(data_content[data_content.index==x]['name'].values)),df_recommnend.index.values))
    df_recommnend['id']=list(map(
        lambda x: int(np.squeeze(data_content[data_content.index==x]['item_id'].values)),df_recommnend.index.values))
    df_recommnend.reset_index()
    
    return df_recommnend.Names.values[:10]

In [140]:
recommend('Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn')

array(['Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn',
       'Tai nghe Bluetooth Lanith i12 TWS 5.0 không dây – Tai nghe không dây kiểu dáng hiện đại, thời thượng - Hàng nhập khẩu – TAI0001',
       'Tai nghe bluetooth không dây F9 True wireless Dock Sạc có Led Báo Pin Kép',
       'Tai Nghe Bluetooth True Wireless AMOI F9 5.0 Cảm Ứng Vân Tay, Dock Sạc có Led Báo Pin Kép - Hàng Nhập Khẩu',
       'Adapter Sạc 2 Cổng Anker PowerPort Mini 12W - A2620 - Hàng Chính Hãng',
       'Adapter Sạc 1 Cổng Anker 18W / 20W PowerPort III Nano Tích Hợp PowerIQ 3.0 - Hàng Chính Hãng',
       'Tai Nghe Bluetooth không dây i12 - TWS, Nhỏ Gọn, Tiện Lợi, Cảm ứng tay -Hàng Chính Hãng',
       'Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2 Trong 1 ]] - Robot - Hàng Chính Hãng',
       'Tai Nghe True Wireless Không Dây QCY T1C - Hàng Chính Hãng',
       'Pin Sạc Dự Phòng Samsung EB-P1100 10000mAh Hỗ Trợ Sạc Nhanh QC 2.0 - Hàng Chính Hãng'],
      dtype=obj

In [173]:
def recommend(content):
    content=data_content[data_content.name==content].index.values[0]
    
    df_recommnend=pd.DataFrame(
        enumerate(vector[content])).drop(0,axis='columns').sort_values(by=1,ascending=False)
    
    df_recommnend['Names']=list(map(
        lambda x: str(np.squeeze(data_content[data_content.index==x]['name'].values)),df_recommnend.index.values))
    df_recommnend['id']=list(map(
        lambda x: int(np.squeeze(data_content[data_content.index==x]['item_id'].values)),df_recommnend.index.values))
    df_recommnend['url']=list(map(
        lambda x: np.squeeze(data_content[data_content.index==x]['url'].values),df_recommnend.index.values))
    df_recommnend['image']=list(map(
        lambda x: np.squeeze(data_content[data_content.index==x]['image'].values),df_recommnend.index.values))
    df_recommnend['rating']=list(map(
        lambda x: int(np.squeeze(data_content[data_content.index==x]['rating'].values)),df_recommnend.index.values))
    df_recommnend['price']=list(map(
        lambda x: int(np.squeeze(data_content[data_content.index==x]['price'].values)),df_recommnend.index.values))
    df_recommnend['brand']=list(map(
        lambda x: np.squeeze(data_content[data_content.index==x]['brand'].values),df_recommnend.index.values))
    df_recommnend = df_recommnend.reset_index()
    df_recommnend = df_recommnend.drop(['index'], axis=1)
    #df_recommnend = df_recommnend.rename(columns={'index', 'Thang do', 'Names', 'id', 'url', 'image', 'rating', 'price'})
    return df_recommnend

In [174]:
x = recommend('Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn')
x


,1,Names,id,url,image,rating,price,brand
0,1.00,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,48102821,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,77000,OEM
1,0.20,Tai nghe Bluetooth Lanith i12 TWS 5.0 không dâ...,79857302,https://tai-nghe-bluetooth-lanith-i12-tws-5-0-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,79000,LANITH
2,0.11,Tai nghe bluetooth không dây F9 True wireless ...,52333193,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,132000,OEM
3,0.11,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...,35373097,https://tai-nghe-bluetooth-true-wireless-amoi-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,135000,AMOI
4,0.09,Adapter Sạc 2 Cổng Anker PowerPort Mini 12W - ...,11477102,https://adapter-sac-2-cong-anker-powerport-min...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,214000,Anker
5,0.09,Adapter Sạc 1 Cổng Anker 18W / 20W PowerPort I...,54017350,https://adapter-sac-1-cong-anker-powerport-iii...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,400000,Anker
6,0.09,"Tai Nghe Bluetooth không dây i12 - TWS, Nhỏ Gọ...",21017060,https://tai-nghe-bluetooth-khong-day-i12-tws-n...,https://salt.tikicdn.com/cache/280x280/ts/prod...,0,79000,OEM
7,0.08,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,57440329,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,149000,Acome
8,0.08,Tai Nghe True Wireless Không Dây QCY T1C - Hàn...,58215473,https://tai-nghe-true-wireless-khong-day-qcy-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,212000,QCY
9,0.08,Pin Sạc Dự Phòng Samsung EB-P1100 10000mAh Hỗ ...,7820533,https://pin-sac-du-phong-samsung-eb-p1100-1000...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,285000,Samsung


In [133]:
x

,1,Names,id,url,image,rating,price
1,1.00,Tai nghe bluetooth không dây F9 True wireless ...,52333193,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,132000
23,0.76,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...,35373097,https://tai-nghe-bluetooth-true-wireless-amoi-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,135000
27,0.17,Adapter Sạc 1 Cổng Anker 18W / 20W PowerPort I...,54017350,https://adapter-sac-1-cong-anker-powerport-iii...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,400000
35,0.17,Adapter Sạc 2 Cổng Anker PowerPort Mini 12W - ...,11477102,https://adapter-sac-2-cong-anker-powerport-min...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,214000
12,0.16,Pin Sạc Dự Phòng Samsung EB-P1100 10000mAh Hỗ ...,7820533,https://pin-sac-du-phong-samsung-eb-p1100-1000...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,285000
15,0.16,Dây cáp sạc đa năng Baseus Rapid 4 in 1 Type-C...,8141868,https://day-cap-sac-da-nang-baseus-rapid-4-in-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,93000
30,0.15,Tai nghe Inpod i12 TWS Bluetooth 5.0 cho iPhon...,35723184,https://tai-nghe-inpod-i12-tws-bluetooth-5-0-c...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,75800
0,0.11,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,48102821,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,77000
4,0.11,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,38458616,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,5090000
21,0.11,[[ KÈM VIDEO ]] - Pin Sạc dự phòng 10000mAh Ro...,51884751,https://pin-sac-du-phong-10000mah-robot-cho-th...,https://salt.tikicdn.com/cache/280x280/ts/prod...,4,129000


In [122]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 1 to 34
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       39 non-null     float64
 1   Names   39 non-null     object 
 2   id      39 non-null     int64  
 3   url     39 non-null     object 
 4   image   39 non-null     object 
 5   rating  39 non-null     int64  
 6   price   39 non-null     int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.4+ KB


In [89]:
import pickle

In [90]:
pickle.dump(df,open('item_list.pkl','wb'))
pickle.dump(vector,open('similarity.pkl','wb'))

In [ ]:
df.head()